In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,27453
2,Huelva,Confirmados PDIA 14 días,1357
3,Huelva,Tasa PDIA 14 días,"264,4347876921878"
4,Huelva,Confirmados PDIA 7 días,624
5,Huelva,Total Confirmados,27627
6,Huelva,Curados,22961
7,Huelva,Fallecidos,359


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  27453.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7134.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 992 personas en los últimos 7 días 

Un positivo PCR cada 469 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,27453.0,624.0,1357.0,513170.0,121.597132,264.434788,126.0
Condado-Campiña,8406.0,315.0,682.0,156231.0,201.624518,436.533082,76.0
Huelva-Costa,16190.0,254.0,562.0,289548.0,87.722934,194.095625,44.0
Huelva (capital),7134.0,145.0,307.0,143837.0,100.808554,213.436042,25.0
Almonte,1339.0,113.0,236.0,24507.0,461.092749,962.990166,25.0
Cartaya,1216.0,73.0,159.0,20083.0,363.491510,791.714385,13.0
Moguer,1020.0,44.0,92.0,21867.0,201.216445,420.725294,9.0
Bollullos Par del Condado,740.0,22.0,40.0,14387.0,152.915827,278.028776,8.0
San Juan del Puerto,418.0,12.0,27.0,9411.0,127.510360,286.898310,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rociana del Condado,471.0,37.0,80.0,7939.0,466.053659,1007.683587,4.0
Almonte,1339.0,113.0,236.0,24507.0,461.092749,962.990166,25.0
Cartaya,1216.0,73.0,159.0,20083.0,363.491510,791.714385,13.0
San Bartolomé de la Torre,280.0,8.0,28.0,3761.0,212.709386,744.482850,3.0
Lucena del Puerto,188.0,5.0,22.0,3261.0,153.327200,674.639681,1.0
Almonaster la Real,62.0,7.0,9.0,1788.0,391.498881,503.355705,0.0
Palos de la Frontera,560.0,26.0,55.0,11742.0,221.427355,468.404020,4.0
Villarrasa,145.0,6.0,10.0,2211.0,271.370421,452.284034,3.0
Condado-Campiña,8406.0,315.0,682.0,156231.0,201.624518,436.533082,76.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,161.0,1.0,7.0,3933.0,25.425884,177.981185,0.0,0.142857
Lucena del Puerto,188.0,5.0,22.0,3261.0,153.327200,674.639681,1.0,0.227273
Punta Umbría,686.0,3.0,13.0,15355.0,19.537610,84.662976,1.0,0.230769
Villalba del Alcor,471.0,1.0,4.0,3366.0,29.708853,118.835413,0.0,0.250000
Aljaraque,977.0,9.0,32.0,21474.0,41.911148,149.017416,2.0,0.281250
San Bartolomé de la Torre,280.0,8.0,28.0,3761.0,212.709386,744.482850,3.0,0.285714
Aracena,499.0,9.0,31.0,8255.0,109.024833,375.529982,1.0,0.290323
Bonares,227.0,8.0,26.0,6060.0,132.013201,429.042904,2.0,0.307692
Aroche,87.0,1.0,3.0,3054.0,32.743942,98.231827,0.0,0.333333
